In [ ]:
!pip install opendatasets

import pandas as pd

df= pd.read_csv('/content/urbansound8k/data.csv')
df.head(5)

import librosa
audio_file_path='/content/urbansound8k/fold10/100648-1-0-0.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

print(librosa_audio_data)

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(librosa_audio_data)

# @title Import Modules
import pandas as pd
import numpy as np
import os, librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from sklearn.metrics import confusion_matrix
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)

mfccs

# @title DataPath
audio_dataset_path = '/content/data'
metadata =  pd.read_csv('/content/urbansound8k/data.csv')
metadata.head()

def mfccExtract(file):
    waveform, sampleRate = librosa.load(file_name)
    features = librosa.feature.mfcc(y = waveform, sr = sampleRate, n_mfcc = 50)
    return np.mean(features, axis = 1)

extractAll = []

for index_num, row in tqdm(metadata.iterrows()):
    # Constructing file path
    file_name = os.path.join(audio_dataset_path, 'fold' + str(row['fold']), row['slice_file_name'])

    # Extracting features and appending them
    features = mfccExtract(file_name)
    extractAll.append([features, row['class']])

featuresDf = pd.DataFrame(extractAll, columns = ['Features', 'Class'])
featuresDf.head()

X=np.array(featuresDf['Features'].tolist())
Y=np.array(featuresDf['Class'].tolist())

labelencoder=LabelEncoder()
Y=to_categorical(labelencoder.fit_transform(Y))

Y

### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
num_labels=Y.shape[1]

from tensorflow.keras.applications import VGG19
from keras.layers import *
base_model = VGG19(weights='imagenet',
                       include_top=False,
                       input_shape=(32, 32, 3))

from tensorflow.keras.models import  Model
# Add classification layers on top of it
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu',input_shape = (50,))(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 10)

test_accuracy=model.evaluate(X_test,Y_test,verbose=0)
print(test_accuracy[1])

historyDf = pd.DataFrame(history.history)

historyDf.loc[:, ['loss', 'val_loss']].plot()

historyDf.loc[:, ['accuracy', 'val_accuracy']].plot()

# Evaluating model
score = model.evaluate(X_test, Y_test)[1] * 100
print(f'Validation accuracy of model : {score:.2f}%')